# Data Fixes

## Imports

In [70]:
import pandas as pd
import numpy as np
import datetime
import numbers
import decimal

Mögliche Verbesserungen an den Daten:

- (Done) Jahreszeit hinzufügen, wann eine Milchleistungsprüfung / Besamung vorgenommen wurde (Juni-Ende August) - (September-Oktober) - (November-Ende Februar) - (März-Ende Mai)

- (Was already Done) Information der Art der Milchleistungsprüfmethode hineinnehmen

- milking_time_morning und milking_time_evening ignorieren -- in Modulus ausschliessen

- (Done) Die Information AR_PESEE_PESCODEALPAGE (ob auf Alp oder nicht) weiter verfeinern, da sehr viele Einträge den Wert 0 haben -> Anstatt 0 mit 350 füllen

- Information der Herde (idhrd_anon) “retten”, z.B. indem man berechnet wie die Milchleistung im Vergleich der mittleren Milchleistung der Herde ist -- noch zu klären

- (Done) Zeilen entfernen, bei welchen die Spalte label nicht ein “CH” drin hat

- milk_yield_msrmt_type (Typ wie der Milk Yield gemessen wurde) möglicherweise ignorieren -- in Modulus ausschliessen

- (Done) Werte mit nsaiet_type == Belegung oder == ET entfernen

- Berechnung Persistenz (Steigung von 1 bis 3 Messung) (Steigung von 3 bis 6 Messung) -> Werte müssten separat importiert werden daher aufgeschoben

In [55]:
# Path where the data ready for the ML analysis is stored
fpath_prepared_data_dir = 'ready_data/ai_basic_all/'

df = pd.read_csv(fpath_prepared_data_dir+'data.csv', sep=',')

print(df.shape)
#print(df.head() )
print(df.columns)

(998174, 41)
Index(['parity', 'nsaiet_date', 'nsaiet_type', 'AI_technician', 'idani_anon',
       'idani_anon_aisire', 'parity_labels', 'mlksmpl_date',
       'milking_time_morning', 'milking_time_evening', 'lab_date', 'DIM',
       'pruefmethode', 'melkmethode', 'milk_yield_24h', 'fat_24h',
       'protein_24h', 'lactose_24h', 'scc_24h', 'urea_24h',
       'AR_PESEE_PESCODEALPAGE', 'idhrd_anon', 'milk_yield_msrmt_type',
       'fat_protein_24h_ratio', 'calving_date', 'calving_ease', 'birth_date',
       'brd_abbr_icar', 'base', 'label', 'ekg', 'epr', 'fkg', 'fpr', 'mkg',
       'per', 'scs', 'healthevents', 'age', 'days_since_calving',
       'days_since_mlksample'],
      dtype='object')


## Clean up 

In [56]:
def remove_columns_to_be_ignored(df, columns_to_ignore):
    """
    Remove all columns to be ignored.
    
    :param df: Pandas dataframe
    :param columns_to_ignore: List of the column names of the columns to ignore
    :return: Pandas dataframe
    """
    return df.drop(labels=columns_to_ignore, axis=1)


def convert_column_to_datetime(df, column, format='%Y-%m-%d', convert_to_int=False):
    """
    Convert a date-like column to a pandas datetime-column.
    
    :param df: Pandas dataframe
    :param column: Column name of the datetime column
    :param format: Format of the date-entry (default: YYYYMMDD)
    :param convert_to_int: Bool value whether the column needs to be converted to an
                           integer column first (default: False)
    :return: Pandas dataframe
    """
    if convert_to_int:
        df[column] = df[column].values.astype(np.int64)
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d', errors='coerce').values
    return df

df = convert_column_to_datetime(df, column='nsaiet_date', convert_to_int=False)
df = convert_column_to_datetime(df, column='mlksmpl_date', convert_to_int=False)
df = convert_column_to_datetime(df, column='lab_date', convert_to_int=False) 
df = convert_column_to_datetime(df, column='calving_date', convert_to_int=False) 
df = convert_column_to_datetime(df, column='birth_date', convert_to_int=False)

print(df.head() )

   parity nsaiet_date nsaiet_type  AI_technician       idani_anon  \
0       1  2012-01-31    Besamung             10  CHE000002135865   
1       2  2013-01-31    Besamung             10  CHE000002135865   
2       2  2013-04-10    Besamung             10  CHE000002135865   
3       3  2014-04-18    Besamung             10  CHE000002135865   
4       5  2016-01-16    Besamung             10  CHE000002135865   

  idani_anon_aisire  parity_labels mlksmpl_date  milking_time_morning  \
0   CHE000027115493              1   2012-01-30               50000.0   
1   CHE000027115493              0   2013-01-19               50000.0   
2   CHE000054877989              1   2013-04-03               50000.0   
3   CHE000039636778              1   2014-03-24               50000.0   
4   CHE000042853985              0   2016-01-14               50000.0   

   milking_time_evening  ...  epr  fkg   fpr    mkg    per    scs  \
0              163000.0  ... -0.1  0.0  0.07 -153.0  108.0  111.0   
1       

## Data Improvements

In [57]:
df['sample_season'] = df.apply(lambda row: 0 if row.mlksmpl_date is None else (1 if row.mlksmpl_date.month >= 6 and row.mlksmpl_date.month <=8 else (2 if row.mlksmpl_date.month >=9 and row.mlksmpl_date.month <= 10 else (3 if row.mlksmpl_date.month >=11 or row.mlksmpl_date.month <=2 else (4 if row.mlksmpl_date.month >= 3 and row.mlksmpl_date.month <= 5 else 0)))), axis = 1)
df['AR_PESEE_PESCODEALPAGE'] = df.apply(lambda row: 350 if row.AR_PESEE_PESCODEALPAGE is None else (350 if row.AR_PESEE_PESCODEALPAGE < 350 else row.AR_PESEE_PESCODEALPAGE), axis = 1)
df = df[df.label.eq('CH')]
df = df[df.nsaiet_type != 'ET']

In [58]:
df.to_csv(fpath_prepared_data_dir+'data.csv', index=False)

In [82]:
print(df[df.sample_season == 1].shape[0])
#print(df[df.nsaiet_type.isin(["Belegung","ET"])].shape[0])
print(df[df.nsaiet_type.isin(["ET"])].shape[0])
df1 = df.drop(['parity_labels'], axis=1)
corrMap = {}

for columnName in df1.columns:
    if isinstance(df1[columnName].iloc[0], numbers.Number):
        myCorr = df['parity_labels'].corr(df1[columnName], method='pearson')
        corrMap[myCorr] = columnName        
        #print(columnName+': '+str(myCorr))
     
for key in sorted(pearsoncorrs):
    print(corrMap[key]+': '+str(key))

195109
0
AI_technician: -0.1059161100147355
milk_yield_24h: -0.10007911149606408
mkg: -0.055811071965679096
ekg: -0.0509425397355961
fkg: -0.0490067287789213
age: -0.04267114262126695
parity: -0.04047877481810854
fat_protein_24h_ratio: -0.022121635892774973
calving_ease: -0.01668457061517801
lactose_24h: -0.015465740604135012
healthevents: -0.007874451667106153
milk_yield_msrmt_type: -0.007762495368695016
per: -0.005761727431111544
milking_time_evening: -0.0032740487797887972
urea_24h: -0.0019487348614256829
milking_time_morning: -0.00030718921321647505
sample_season: 0.005588093764825396
days_since_mlksample: 0.0060189259985440864
scc_24h: 0.006316029662490601
scs: 0.007274577954998785
fpr: 0.00974408441947971
AR_PESEE_PESCODEALPAGE: 0.012268519325749494
fat_24h: 0.016659447301045113
epr: 0.019585598885896078
DIM: 0.05617347486686405
days_since_calving: 0.0583797114475067
protein_24h: 0.06052924291374965
